#### Best Params
>  **Obtained Via Hyper Parameter Optimization**

>
>    **For RELIANCE.NS only  Need to perform Hyper-Parameter Optimization for other Stocks.** 

In [ ]:
# Best Hyper-Paramameter Values for RELIANCE.NS Stock.
best_params = {
    'hidden_size': 765,
    'num_layers': 1,
    'dropout': 0.09706022396783648,
    'learning_rate': 0.0014995936033621617,
    'weight_decay': 0.0009768589023152823,
    'batch_norm': False,
    'cell_dropout': 0.04166033473442454,
    'optimizer': 'adamw',
    'lr_scheduler': 'cosine',
    'lr_T_max': 49,
    'loss_function': 'huber',
    'delta': 0.1403443377256188
}

####  1. Select Stock Ticker

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Set the ticker symbol
ticker = "INFY.NS"

#### 2. Select appropriate device

In [ ]:
import torch

# Select appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


#### 3. Run the CUDA check

In [4]:
from Config.check_cuda_config import check_cuda_configuration

# Run the CUDA check
check_cuda_configuration()

CUDA is available with 1 device(s).
Current device: 0 - NVIDIA GeForce RTX 3050 Laptop GPU
CUDA version: 12.1
Total GPU memory     : 4.29444301 GB
Currently allocated  : 0.00000000 GB
Currently reserved   : 0.00000000 GB


True

#### 4. Fetch stock data

In [5]:
import os

from DataPipeline.data_fetcher import fetch_daily_data_ist

# Create the path if it doesn't exist
raw_data_dir = "Data/RawData"

# Fetch stock data
data = fetch_daily_data_ist(ticker)

# Construct filename and full path
filename = f"{ticker[:-3]}.csv"
filepath = os.path.join(raw_data_dir, filename)

# Save to CSV
data.to_csv(filepath)
print(f"Fetched raw data saved to: {filepath}")
data.head()

Fetched raw data saved to: Data/RawData\INFY.csv


,Open,High,Low,Close,Volume,Dividends,Stock Splits
2010-01-04,232.496413,234.277995,231.186955,232.728027,4069264,0.0,0.0
2010-01-05,233.387150,234.340292,231.614486,233.507416,6895528,0.0,0.0
2010-01-06,238.999118,238.999118,229.387496,230.100128,6817288,0.0,0.0
2010-01-07,230.411960,230.411960,224.318938,224.929138,10892600,0.0,0.0
2010-01-08,224.390221,224.773265,218.956396,219.508682,12649312,0.0,0.0


#### 5. Calculate Technical Indicators

In [6]:
from DataPipeline.technical_indicators import TechnicalIndicators

# Calculate technical indicators
indicators = TechnicalIndicators(data)
indicators_data = indicators.calculate_all()

#### 6. Data Splitting

In [7]:
print(f"\nProcessed data shape after indicators: {indicators_data.shape}")

# Split data into train and validation (85%) and test (15%)
total_size = len(indicators_data)
train_size = int(0.85 * total_size)

train_data = indicators_data[:train_size].copy()
test_data = indicators_data[train_size:].copy()

print(f"\nData split sizes:")
print(f"\nTrain: {len(train_data)}, Test: {len(test_data)}")


Processed data shape after indicators: (3802, 81)

Data split sizes:

Train: 3231, Test: 571


#### 7. Dropping NaN Columns and Rows

In [8]:
from DataPipeline.data_cleaning import drop_all_null_columns

# Drop all-null columns
train_data_ready, dropped_columns = drop_all_null_columns(train_data)
test_data_ready, _ = drop_all_null_columns(test_data)

print(f"\nDropped columns due to nulls: {dropped_columns}")
print(f"Final shapes - Train: {train_data_ready.shape}, Test: {test_data_ready.shape}")

No columns with all null values found.
No columns with all null values found.

Dropped columns due to nulls: []
Final shapes - Train: (3204, 81), Test: (570, 81)


#### 8. Data Preparation

In [9]:
from Optimization.prepare_data_for_hpo import prepare_data_for_hpo

# Prepare data for LSTM (only train loader, no val)
data, train_loader, val_loader, X_scaler, y_scaler, feature_names, num_features = prepare_data_for_hpo(
    Modelling_data=train_data_ready,
    model_type="LSTM",
    batch_size=64,
    seq_length=30
)

#### 9. Model Training

In [10]:
import time

from Models.create_model_with_params import create_model_with_params
from Models.train_final_model import train_final_model

# Record start time
start_time = time.time()

# Train the model
Trained_Model = train_final_model(
    best_params=best_params,
    model_type="lstm",
    train_loader=train_loader,
    val_loader=val_loader,  # explicitly no validation
    num_features=num_features,
    scaler_y=y_scaler,
    input_features=feature_names,
    data=data,
    output_dir="Results",
)

# Record end time
end_time = time.time()
elapsed = end_time - start_time
print(f"Total training time: {elapsed:.2f} seconds")


Training final model with best hyperparameters...
 {'hidden_size': 765, 'num_layers': 1, 'dropout': 0.09706022396783648, 'learning_rate': 0.0014995936033621617, 'weight_decay': 0.0009768589023152823, 'batch_norm': False, 'cell_dropout': 0.04166033473442454, 'optimizer': 'adamw', 'lr_scheduler': 'cosine', 'lr_T_max': 49, 'loss_function': 'huber', 'delta': 0.1403443377256188}
Epoch 1/500 - Train Loss: 0.02977309, Val Loss: 0.16111812 (Best Model Saved)
Epoch 2/500 - Train Loss: 0.00489675, Val Loss: 0.04369849 (Best Model Saved)
Epoch 3/500 - Train Loss: 0.00131868, Val Loss: 0.07206188
Epoch 4/500 - Train Loss: 0.00192032, Val Loss: 0.01996631 (Best Model Saved)
Epoch 5/500 - Train Loss: 0.00108112, Val Loss: 0.03384035
Epoch 6/500 - Train Loss: 0.00096074, Val Loss: 0.00855320 (Best Model Saved)
Epoch 7/500 - Train Loss: 0.00077805, Val Loss: 0.02292035
Epoch 8/500 - Train Loss: 0.00160311, Val Loss: 0.04674455
Epoch 9/500 - Train Loss: 0.00054935, Val Loss: 0.00641991 (Best Model Sav

#### 10. Test Set Predictions

In [ ]:
# ====================================
# Imports
# ====================================
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset

# Custom modules
from Models.evaluate_model import evaluate_model
from Optimization.select_features_for_model import select_features_for_model
from PlotScripts.get_time_series_comparison_plot import time_series_comparison_plot
from PlotScripts.get_scatter_plot import scatter_plot


# ====================================
# Step 1: Data Preparation
# ====================================
df = test_data_ready.copy()  # Work on a copy to preserve original
valid_features = select_features_for_model(df, "lstm")  # Select features suitable for LSTM
seq_length = 30  # Length of input sequences


# ====================================
# Step 2: Feature & Target Scaling
# ====================================
X_all = X_scaler.transform(df[valid_features].values)   # Feature scaling
y_all = df["Close"].values.reshape(-1, 1)               # Target variable
y_all_scaled = y_scaler.transform(y_all)                # Target scaling


# ====================================
# Step 3: Create Sliding Windows (Sequence Generation)
# ====================================
X_windows = []  # Sequences of input features
y_targets = []  # Corresponding next-day targets

for end_ix in range(seq_length, len(X_all)):
    start_ix = end_ix - seq_length
    X_windows.append(X_all[start_ix:end_ix])
    y_targets.append(y_all_scaled[end_ix])

X_tensor = torch.tensor(X_windows, dtype=torch.float32)
y_tensor = torch.tensor(y_targets, dtype=torch.float32)


# ====================================
# Step 4: DataLoader Creation
# ====================================
dataset = TensorDataset(X_tensor, y_tensor)
loader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)


# ====================================
# Step 5: Model Evaluation
# ====================================
metrics = evaluate_model(Trained_Model, data_loader=loader)

# ====================================
# Step 6: Model Inference for Plotting
# ====================================
Trained_Model.eval()  # Set model to evaluation mode
X_tensor = X_tensor.to(device)

with torch.no_grad():
    y_pred_scaled = Trained_Model(X_tensor).view(-1, 1).cpu().numpy()

# Inverse scale to get actual price values
y_pred = y_scaler.inverse_transform(y_pred_scaled).flatten()
y_true = y_all[seq_length:].flatten()
dates = df.index[seq_length:]


# ====================================
# Step 7: Visualization
# ====================================
time_series_comparison_plot(
    targets_original=y_true,
    predictions_original=y_pred,
    model_type="LSTM",
    output_dir="Results",
    phase="Testing"
)
scatter_plot(
    targets_original=y_true,
    predictions_original=y_pred,
    model_type="LSTM",
    output_dir="Results",
    phase="Testing"
)

# ====================================
# Step 8: Store and Print Results
# ====================================
results = {
    f"{ticker}_predictions": y_pred,
    f"{ticker}_actuals": y_true,
    **{f"{ticker}_{k}": v for k, v in metrics.items()},
}



Model Evaluation Metrics:
MSE                 : 0.017329
RMSE                : 0.131639
MAE                 : 0.094420
R2                  : 0.986200
MAPE                : 1.670436
EXPLAINED_VARIANCE  : 0.986389
MAX_ERROR           : 0.626125


####  11. Generate Forecast 

In [13]:
# Import the forecasting function from the utility module
from Utils.get_forecast import generate_forecast

# Create a copy of the test data to avoid modifying the original dataset
forecast_data = test_data_ready.copy()

# Generate forecast using the trained LSTM model
generate_forecast(
    forecast_data=forecast_data,  # Input data prepared for forecasting
    Trained_Model=Trained_Model,  # The trained model used for prediction
    X_scaler=X_scaler,  # Scaler used to normalize input features
    y_scaler=y_scaler,  # Scaler used to denormalize output predictions
    seq_length=30,  # Number of time steps to look back for each prediction
    ticker=ticker,  # Ticker symbol for the stock (or identifier for the time series)
    model="lstm",  # Type of model to be used ("lstm" in this case)
)

Predicted Close Price of INFY.NS for 2025-06-02: 1566.3815
